# Make patches from labels 

This notebook shows how to make patches from labels

For the purpose of this notebook we use the following definitions

1.  Label - set of annotations for an image.
2.  Patch - a crop of the 'label' that is ready to be an input to train a neural network.

Patches are essentially crops from the label. The size of the patch is the input size of the network.  

There are different strategies to make patches.  In this notebook we create a ordered set of patches to cover the entire image.

For example if the image is 512 by 512 and the patch size is 256 by 256 we create 4 patches.  If the image is say 400 by 400 we still create 4 256 by 256 patches but they will have an overlap. 


## Load the image and previous segmentations

In [2]:
from skimage.io import imread, imsave
import napari
from tnia.deeplearning.dl_helper import make_patch_directory
import os
from pathlib import Path
import json
import numpy as np

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_01_12_plant_roots')
name = '21_force_filtered'
im = imread(os.path.join(parent_path / (name+".tif")))

#labels = np.zeros(im.shape, dtype=np.uint8)
labels = imread(os.path.join(parent_path / (name+"_labeled_.tif")))

## Create folders for input and ground truth patches

When we train a neural network we usually use small patches.  So create a directory for input (the original image) and ground truth (the labels) patches, then save the patches.

In [3]:
train_path= parent_path / 'labels'
image_patch_path =  train_path / 'input0'
label_patch_path =  train_path / 'ground truth0'

if not os.path.exists(train_path):
    os.mkdir(train_path)
if not os.path.exists(image_patch_path):
    os.mkdir(image_patch_path)
if not os.path.exists(label_patch_path):
    os.mkdir(label_patch_path)

patch_base_name = name
axes = 'YXC'
sub_sample = 1

make_patch_directory(1, 1, train_path)

# Load the existing JSON data which is created when making the patch directory and append addition information to it
json_file = train_path / "info.json"

with open(json_file, 'r') as infile:
    data = json.load(infile)

# add the sub_sample information to the JSON file
data['sub_sample'] = sub_sample 
data['axes'] = axes

# Write the modified data back to the JSON file
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)


# Make a utility to calculate the number of patches and step size 

If the image size is a multiple of the patch size (ie image size = 1024, patch size = 256) things work out nice, if not we need some overlap between patches (it we want to cover the entire image).  Below utility calculate the number of division and step size that will be used to create patches

In [8]:
import math

def get_num_divisions(shape, patch_size):
    num_divisions = []
    step = []

    for d in shape:
        num_divisions_=math.ceil(d/patch_size)

        if num_divisions_ > 1:
            remainder = num_divisions_*patch_size-d
            overlap = remainder/(num_divisions_-1)
            step_ = math.floor(patch_size-overlap)
            num_divisions.append(num_divisions_)
            step.append(step_)
        else:
            num_divisions.append(1)
            step.append(0)
    return num_divisions, step



## Crop patches and save

In [100]:
from skimage.io import imsave

patch_size = (256, 256)

num_divisions, step = get_num_divisions(im.shape, 256)

for i in range(num_divisions[0]):
    for j in range (num_divisions[1]):
        print(i, j)
        sx = i*step[0]
        sy = j*step[1]
        ex = sx+256
        ey = sy+256
        print(sx, sy, ex, ey)

        im_patch = im[sx:ex, sy:ey]
        label_patch = labels[sx:ex, sy:ey]
        im_patch_name = os.path.join(image_patch_path, patch_base_name+f"_{i}_{j}.tif")
        label_patch_name = os.path.join(label_patch_path, patch_base_name+f"_{i}_{j}.tif")
        
        print(im_patch_name)
        print(label_patch_name)
        print(im_patch.shape)
        print(label_patch.shape)
        print('-----------------')
        imsave(im_patch_name, im_patch)
        imsave(label_patch_name, label_patch)
        

0 0
0 0 256 256
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\input0\21_force_filtered_0_0.tif
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_0.tif
(256, 256)
(256, 256)
-----------------
0 1
0 254 256 510
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\input0\21_force_filtered_0_1.tif
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_1.tif
(256, 256)
(256, 256)
-----------------
0 2
0 508 256 764
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\input0\21_force_filtered_0_2.tif
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_2.tif
(256, 256)
(256, 256)
-----------------
0 3
0 762 256 1018
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\input0\21_force_filtered_0_3.tif
D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_forc

C:\Users\bnort\AppData\Local\Temp\ipykernel_37308\2875118385.py:25: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_0.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_37308\2875118385.py:25: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_1.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_37308\2875118385.py:25: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_2.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_37308\2875118385.py:25: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_12_plant_roots\labels\ground truth0\21_force_filtered_0_3.tif is a low contrast image
  imsave(label_patch_name, label